In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from src.database import read_sql

In [ ]:
sql_sickness_abbr = '''
    SELECT neu_kurz as sickness_abbr, string_agg(sdienst_lang, ', ') as description FROM spxdb_archiv_2023.sdienst_mapping
    WHERE sdienst_lang ILIKE('%krank%') AND sdienst_lang NOT ILIKE('%nicht krank%')
    GROUP BY sickness_abbr
'''
read_sql(sql_sickness_abbr)

In [ ]:
sql_sickness_plan = f'WITH mapping AS ({sql_sickness_abbr})' + '''
    SELECT *
    FROM mapping JOIN spxdb_archiv_2023.monatsplan_adapted AS plan
    ON mapping.sickness_abbr = plan.sdienst_kurz_adapted
    WHERE plan.mpebene = 1 -- filter for "ist" (vs. e.g. "soll")
'''
read_sql(sql_sickness_plan + ' LIMIT 10')

In [ ]:
sickness_count = read_sql(f'''
    WITH plan AS ({sql_sickness_plan})
    SELECT
        DATE_TRUNC('week', (DATE '1990-01-01' + (tag_f * INTERVAL '1 day'))) AS week_start,
        COUNT(*) as sickness_entries
     FROM plan GROUP BY week_start;
''')

plt.figure(figsize=(12, 6))
plt.plot(sickness_count['week_start'], sickness_count['sickness_entries'])
plt.title('Wöchentliche Eintragungen von Krankheit')
plt.xlabel('Kalenderwoche')
plt.ylabel('Anzahl')